#  WORKS!!!

## combined w/ filenames

In [3]:
import os
import pandas as pd

# Path to the folder containing CSV files
folder_path = '../csvs/csv-behavior_sni-mMORp-DREADD_sex/A_Baseline_NoSNI'

group = 'Male_Group1'

# List to store all individual dataframes
dfs = []

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        # Exclude the first column
        df = df.iloc[:, 1:]
        # Rename the columns to avoid duplicates
        df.columns = [f'{col}_{file_name}' for col in df.columns]
        # Add the dataframe to the list
        dfs.append(df)

# Concatenate all dataframes in the list along the columns axis
combined_df = pd.concat(dfs, axis=1)

# Save the combined dataframe to a new CSV file
combined_df.to_csv(f'../csvs/Condition3_FormalinDrug/{group}_filenamed.csv', index=False)


## combined w/ behaviors rearranged 

In [14]:
import os
import pandas as pd

#### STEP 1 Combine Indiviual Files

#### CHANGE ACCORDINGLY => Path to the folder containing CSV files // Group name
group = 'Male_Experimental'
folder_path = f'../csvs/fig4_oprm1-reexpress_morphine/{group}'

dfs = []
max_rows = 0

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path, index_col=0)
        # Update the maximum number of rows
        max_rows = max(max_rows, df.shape[0])
        # Add the dataframe to the list
        dfs.append(df)

# Align the rows by filling missing rows with NaN
aligned_dfs = []
for df in dfs:
    rows_to_add = max_rows - df.shape[0]
    if rows_to_add > 0:
        extra_rows = pd.DataFrame([[float('nan')] * df.shape[1]] * rows_to_add, index=range(df.shape[0], max_rows), columns=df.columns)
        df = pd.concat([df, extra_rows])
    aligned_dfs.append(df)

combined_df = pd.concat(aligned_dfs, axis=1)
combined_df.to_csv(f'../csvs/fig4_oprm1-reexpress_morphine/{group}_separated.csv')

#### STEP 2 Rearrange By Behavior

combined_df = pd.read_csv(f'../csvs/fig4_oprm1-reexpress_morphine/{group}_separated.csv', index_col=0)

# Dictionary to store columns for each header value ('0', '1', '2', '3', '4', '5')
header_columns = {}

# Iterate over the columns
for col in combined_df.columns:
    header = col.split('_')[0]  # Extract the header value
    if header not in header_columns:
        header_columns[header] = []  # Initialize an empty list for the header value
    header_columns[header].append(col)  # Append the column to the corresponding header value

# List to store rearranged columns
rearranged_columns = []

# Iterate over the header values and concatenate columns for each header value
for header in sorted(header_columns.keys(), key=lambda x: float(x) if x.replace('.', '').isdigit() else x):
    rearranged_columns.extend(header_columns[header])

# Create a new dataframe with rearranged columns
rearranged_df = combined_df[rearranged_columns]
rearranged_df.to_csv(f'../csvs/fig4_oprm1-reexpress_morphine/{group}_rearranged.csv')